# PURE-TF (1.13.0rc2) FaceAttribute

In [ ]:
import tensorflow as tf
import numpy as np
import random
import os

tf.enable_eager_execution()
AUTOTUNE = tf.data.experimental.AUTOTUNE
NUM_IMGS = 202599

## 1. Dataset

In [ ]:
def load_labels(labels_path):
    f = open(labels_path)
    # line 1: number of images
    num_imgs = int(f.readline())
    # line 2: attribute names, 40 in total
    attr_names = f.readline().split()
    # line 3 to end: 00xx.jpg -1 1 -1 1 ...
    labels = []
    for i in range(num_imgs):
        labels.append(list(map(np.float32, f.readline().split()[1:])))
    labels = np.array(labels)
    labels[labels<0] = 0
    return labels

def load_imgs(imgs_dir):
    img_paths = os.listdir(imgs_dir)
    img_paths.sort()
    for i in range(len(img_paths)):
        img_paths[i] = os.path.join(imgs_dir,img_paths[i])
    return img_paths

def preprocess(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    # uint8 range: [0,255]
    img = tf.image.resize(img, [192, 160])
    # new range: [-128,127]
    img -= 128
    img = tf.image.convert_image_dtype(img, tf.int8, saturate=True)
    return img    

def parse(x):
    result = tf.parse_tensor(x, out_type=tf.int8)
    result = tf.reshape(result, (192,160,3))
    return result

if os.path.exists('../dataset/tfrec') == False:
    os.mkdir('../dataset/tfrec')
    imgs_dir  = '../dataset/img_align_celeba'
    img_paths = load_imgs(imgs_dir)
    ds_imgs = tf.data.Dataset.from_tensor_slices(img_paths).map(preprocess)
    ds_imgs_serialized = ds_imgs.map(tf.serialize_tensor)
    tfrec = tf.data.experimental.TFRecordWriter('../dataset/tfrec/imgs.tfrec')
    tfrec.write(ds_imgs_serialized)
else: 
    ds_imgs = tf.data.TFRecordDataset('../dataset/tfrec/imgs.tfrec')
    ds_imgs = ds_imgs.map(parse, num_parallel_calls=AUTOTUNE)

labels_path = '../dataset/list_attr_celeba.txt'
ds_labels = tf.data.Dataset.from_tensor_slices(load_labels(labels_path))
ds_celeba = tf.data.Dataset.zip((ds_imgs, ds_labels))
ds_celeba = ds_celeba.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=8192*4))
ds_celeba = ds_celeba.batch(32).prefetch(AUTOTUNE)


## 2. Train with Keras 

In [ ]:
mnet = tf.keras.applications.mobilenet.MobileNet(input_shape=(192,160,3),alpha=0.5,include_top=False,weights=None,pooling='avg')
mnet = tf.keras.Sequential([mnet,tf.keras.layers.Dense(40,activation='sigmoid',name='top_dense')], name='mnet_050_faceattr')
mnet.summary()

mnet.compile(optimizer=tf.keras.optimizers.RMSprop(),
            loss='binary_crossentropy',
            metrics=['binary_accuracy'])

mnet.fit(ds_celeba, epochs=5, steps_per_epoch=20, verbose=2)

## 3. Train with pure TF

### 3.1 Build a mobilenet